In [3]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamelog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time

time.sleep(1)
gamelog = leaguegamelog.LeagueGameLog(season='2023-24', player_or_team_abbreviation='T')
df = gamelog.get_data_frames()[0]

df['WIN'] = df['WL'].map({'W': 1, 'L': 0})
features = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
X = df[features]
y = df['WIN']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lda = LinearDiscriminantAnalysis()
svm = SVC(kernel='rbf', probability=True, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

lda.fit(X_train_scaled, y_train)
svm.fit(X_train_scaled, y_train)
rf.fit(X_train_scaled, y_train)
gb.fit(X_train_scaled, y_train)

voting_clf = VotingClassifier(estimators=[
    ('lda', lda),
    ('svm', svm),
    ('rf', rf),
    ('gb', gb)
], voting='soft')

voting_clf.fit(X_train_scaled, y_train)

models = {'LDA': lda, 'SVM': svm, 'Random Forest': rf, 'Gradient Boosting': gb, 'Voting Ensemble': voting_clf}

for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    print(f"\n{name} Results:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))



LDA Results:
Accuracy: 0.8678861788617886
Confusion Matrix:
 [[214  32]
 [ 33 213]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87       246
           1       0.87      0.87      0.87       246

    accuracy                           0.87       492
   macro avg       0.87      0.87      0.87       492
weighted avg       0.87      0.87      0.87       492


SVM Results:
Accuracy: 0.8495934959349594
Confusion Matrix:
 [[212  34]
 [ 40 206]]
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.86      0.85       246
           1       0.86      0.84      0.85       246

    accuracy                           0.85       492
   macro avg       0.85      0.85      0.85       492
weighted avg       0.85      0.85      0.85       492


Random Forest Results:
Accuracy: 0.8028455284552846
Confusion Matrix:
 [[193  53]
 [ 44 202]]
Classification Report:
          